In [1]:
!pip install sentence_transformers
!pip install langchain
!pip install langchain_community
!pip install chromadb

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 227.1/227.1 kB 4.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 997.8/997.8 kB 16.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 394.9/394.9 kB 23.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 149.1/149.1 kB 8.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 54.0/54.0 kB 3.8 MB/s eta 0:00:00
  Attempting uninstall: packaging
    Found existing installation: packaging 21.3
    Uninstalling packaging-21.3:
      Successfully uninstalled packaging-21.3
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
cudf 24.8.2 requires cubinlinker, which is not installed.
cudf 24.8.2 requires cupy-cuda11x>=12.0.0, which is not installed.
cudf 24.8.2 requires ptxcompiler, which is not installed.
cuml 24.8.0 requires cupy-cuda11x>=12.0.0, which is not installed.
dask-cudf 24.8

In [2]:
from langchain.text_splitter import RecursiveCharacterTextSplitter
from sentence_transformers import SentenceTransformer
from langchain.vectorstores import Chroma
from langchain.embeddings import HuggingFaceEmbeddings
from transformers import GPT2LMHeadModel, GPT2Tokenizer
from langchain_core.runnables import RunnablePassthrough
from langchain_core.prompts import PromptTemplate
from langchain_core.output_parsers import StrOutputParser
import os

/opt/conda/lib/python3.10/site-packages/sentence_transformers/cross_encoder/CrossEncoder.py:11: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm, trange


In [3]:
SANAD = r'/kaggle/input/sanad-dataset'
Combined = os.path.join(SANAD, '/kaggle/working/combined_text.txt')

In [4]:
with open(Combined, 'w', encoding='utf-8') as outfile:
    for folder_name in sorted(os.listdir(SANAD)):
        folder_path = os.path.join(SANAD, folder_name)

        if os.path.isdir(folder_path):
            outfile.write(f'\n\n=== Content from Folder: {folder_name} ===\n\n')

            for filename in sorted(os.listdir(folder_path)):
                if filename.endswith('.txt'):
                    file_path = os.path.join(folder_path, filename)
                    with open(file_path, 'r', encoding='utf-8') as infile:
                        outfile.write(infile.read())
                        outfile.write("\n")  


In [5]:
text = r'/kaggle/working/combined_text.txt'

In [6]:
class Document:
    def __init__(self, page_content, metadata=None):
        self.page_content = page_content
        self.metadata = metadata if metadata is not None else {}

In [7]:
with open(text, 'r', encoding='utf-8') as file:
    text_content = file.read()

# Initialize the text splitter
text_splitter = RecursiveCharacterTextSplitter(chunk_size=3450, chunk_overlap=250)

# Create a list of Document objects with metadata
documents = [Document(page_content=text_content, metadata={"source": text})]

# Split the text into smaller chunks
split_documents = text_splitter.split_documents(documents)


print(len(split_documents))

40593


In [8]:
class Embedding:
    def __init__(self):
        self.model_name = 'asafaya/bert-base-arabic'

        self.model = SentenceTransformer(self.model_name)

    def embed_documents(self, docs):
        embeddings = self.model.encode(docs)
        return embeddings.tolist()

    def embed_query(self, query):
        return self.model.encode(query).tolist()

In [9]:
embed_model = Embedding()

config.json:   0%|          | 0.00/491 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/445M [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/62.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/334k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

/opt/conda/lib/python3.10/site-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


In [10]:
hf_embeddings = HuggingFaceEmbeddings(model_name=embed_model.model_name)
vector_database = Chroma.from_documents(documents=split_documents, embedding=hf_embeddings)
retriever = vector_database.as_retriever(search_type='similarity', search_kwargs={'k': 3})

/opt/conda/lib/python3.10/site-packages/langchain_core/_api/deprecation.py:151: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 0.3.0. An updated version of the class exists in the langchain-huggingface package and should be used instead. To use it run `pip install -U langchain-huggingface` and import as `from langchain_huggingface import HuggingFaceEmbeddings`.
  warn_deprecated(


In [11]:
retrieved_docs = retriever.invoke(' أعلنت منظمة التجارة الخارجية اليابانية «جيترو» نجاح مشاركتها في الدورة ال36 لأسبوع جيتكس للتكنولوجيا الذي استضافته دبي في المركز التجاري العالمي الأسبوع الماضي، حيث قدمت اليابان مجموعة من الابتكارات العلمية الجديدة في جناحها الذي استقبل حوالي 25 ألف زائر، ومستثمر.وعرضت الشركات اليابانية في دورة جيتكس 2016، ابتكارات سبع شركات تكنولوجيا ناشئة في مجالات متعددة، أهمها الطباعة ثلاثية الأبعاد التي تعرض من خلال مختبرات «بونساي» الشهيرة، و«مارس إليكتريك»، إلى جانب حلول الشبكات التكنولوجية من خلال مشاركة أجهزة «سيريفو»، وأجهزة عرض الفيديو عالية الدقة وأجهزة القياس الرياضي')

print(retrieved_docs[0].page_content)

دبي:«الخليج» تستعد منظمة التجارة الخارجية اليابانية «جيترو» للمشاركة في الدورة ال36 لأسبوع جايتكس للتكنولوجيا الذي تستضيفه دبي في المركز التجاري العالمي بتاريخ 16 ولغاية 20 أكتوبر الجاري، حيث يقدم جناحها مجموعة من الابتكارات العلمية الفريدة التي تتفوق فيها اليابان عالميا، والتي ستشكل مفاجآت مذهلة بالنسبة لزوّار هذا المعرض السنوي المهم. وضمن المشاركة اليابانية المتميزة، تعرض سبع شركات تكنولوجيا معروفة في طوكيو في أسبوع «جايتكس» آخر منتجاتها واختراعاتها في مجالات متعددة، أهمها الطباعة ثلاثية الأبعاد التي تعرض من خلال مختبرات «بونساي» الشهيرة، و«مارس إليكتريك»، إلى جانب حلول الشبكات التكنولوجية من خلال مشاركة أجهزة «سيريفو»، وأجهزة عرض الفيديو عالية الدقة من «لايف ويدج»، وأجهزة القياس الرياضي من «إكسون».وتتواجد الروبوتات اليابانية الشهيرة في جناح «جيترو»، حيث تعرض شركة «مارس إليكتريك»، المتخصصة في الروبوتات الحديثة، تكنولوجيا جديدة للروبوتات المساندة للخدمات الإنسانية، وهي روبوتات ناطقة يمكن مشاهدتها والتفاعل معها مباشرة.فيما يتوقع أن يلفت اختراع ياباني لجهاز جديد يجنب سائقي السيارات حواد

In [12]:
template = '''
أنت بوت ذكاء اصطناعي تم تطويره من قبل البشر، ومهمتك هي الإجابة على أسئلة المستخدمين بناءً على المعرفة المقدمة في السياق أدناه.

ستكون الأسئلة باللغة العربية، ويجب أن تكون الإجابات أيضًا باللغة العربية.  
السياق المقدم يتضمن سبع فئات: [الثقافة، المالية، الطب، السياسة، الدين، الرياضة، والتقنية].  
حاول تقديم إجابة باستخدام المعلومات المتاحة في السياق. إذا لم تجد الإجابة بشكل واضح في السياق، اتبع الخطوات التالية:

    1. اعتذر للمستخدم.
    2. أبلغه أنك لا تعرف إجابة سؤاله بناءً على المعلومات المتاحة.
    3. اشرح له أنك متخصص فقط في نماذج اللغة الكبيرة (LLMs).
    4. اسأله إذا كان لديه أي أسئلة أخرى.

في نهاية الإجابة، اشكر المستخدم واطلب منه إذا كان لديه أي استفسارات أخرى.

الإجابة يجب أن تكون مفصلة ولا تقل عن 300 كلمة.

إذا سُئلت عن نفسك، أبلغ المستخدم أنك روبوت دردشة تم تطويره من قبل البشر ولست إنسانًا.

المعرفة المتاحة لك:  
{context}

السؤال: {question}

الإجابة:


'''

In [13]:
def format_docs(docs):
    return "\n\n".join(docs) 

In [14]:
aragpt2_model_name = "aubmindlab/aragpt2-medium"
tokenizer = GPT2Tokenizer.from_pretrained(aragpt2_model_name)
aragpt2_model = GPT2LMHeadModel.from_pretrained(aragpt2_model_name)

vocab.json:   0%|          | 0.00/1.94M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/1.50M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/4.52M [00:00<?, ?B/s]

config.json:   0%|          | 0.00/843 [00:00<?, ?B/s]

/opt/conda/lib/python3.10/site-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


model.safetensors:   0%|          | 0.00/1.50G [00:00<?, ?B/s]

In [15]:
def generate_aragpt2_response(context, question):
    input_text = f"{context}\n\nالسؤال: {question}\nالإجابة:"
    
    # Ensure the padding token is set
    tokenizer.pad_token = tokenizer.eos_token  # Set the EOS token as the padding token
    
    # Tokenize the input text with padding
    input_ids = tokenizer.encode(input_text, return_tensors='pt', padding=True)
    
    # Generate response with repetition penalty and attention mask
    output = aragpt2_model.generate(
        input_ids=input_ids,
        max_length=500,
        num_return_sequences=1,
        repetition_penalty=1.2,
        pad_token_id=tokenizer.eos_token_id,  # Use EOS as the padding token
        attention_mask=(input_ids != tokenizer.pad_token_id).long()
    )
    
    # Decode generated text
    generated_text = tokenizer.decode(output[0], skip_special_tokens=True)
    return generated_text


In [16]:
custom_rag_prompt = PromptTemplate.from_template(template)

In [17]:
def rag_chain(context, question):
    formatted_context = format_docs([context]) 
    answer = generate_aragpt2_response(formatted_context, question)
    return answer

In [18]:
example_context = "هنا يوجد مثال على السياق الذي يحتوي على معلومات ذات صلة بسؤال المستخدم"

In [19]:
question ="اقترب نادي مانشستر يونايتد من ضم مدافع نادي فولهام كريس سمولينغ (20 عاماً) مقابل 8 ملايين جنيه بعد موافقة إدارة النادي على بيعه نهاية الموسم الحالي"
answer = rag_chain(example_context, question)
print(answer)
     

هنا يوجد مثال على السياق الذي يحتوي على معلومات ذات صلة بسؤال المستخدم

السؤال: اقترب نادي مانشستر يونايتد من ضم مدافع نادي فولهام كريس سمولينغ (20 عاماً) مقابل 8 ملايين جنيه بعد موافقة إدارة النادي على بيعه نهاية الموسم الحالي
الإجابة: في الوقت الذي كان فيه فريق نيوكاسل يسعى لضم اللاعب ، كانت هناك أنباء عن رغبة نادي ليفربول الإنجليزي في التعاقد مع المدافع الشاب.وذكرت صحيفة " ديلي ميل " البريطانية أن ليفربول رفض عرضا بقيمة 10 مليون جنيه إسترليني للتخلي عن لاعبه البالغ من العمر 19 عاما.وأشارت الصحيفة إلى أن ليفربول لم يكن مستعدا لدفع هذا المبلغ الضخم ، خاصة وأن الفريق الإنجليزي لا يزال يبحث عن مهاجم جديد لتعويض رحيل المهاجم الهولندي رود فان نيستلروي.وكان سوانزي سيتي قد تقدم بعرض قيمته 5 ر 7 مليون جنيه إسترليني للتعاقد مع سمولينغ ، لكن إدارة النادي رفضت العرض.


In [20]:
question_2 ="؟عندما التقى حمد سعيد سلطان الشامسي سفير الدولة لدى بيروت، أمس، وائل أبو فاعور وزير الصحة اللبناني. وناقش الطرفان خلال اللقاء الذي عقد بمقر وزارة الصحة اللبنانية، ماذا ناقش "
answer_2 = rag_chain(example_context, question_2)
print(answer_2)
     

هنا يوجد مثال على السياق الذي يحتوي على معلومات ذات صلة بسؤال المستخدم

السؤال: ؟عندما التقى حمد سعيد سلطان الشامسي سفير الدولة لدى بيروت، أمس، وائل أبو فاعور وزير الصحة اللبناني. وناقش الطرفان خلال اللقاء الذي عقد بمقر وزارة الصحة اللبنانية، ماذا ناقش 
الإجابة: ، كما تم التطرق إلى موضوع الأدوية التي يتم شراؤها من لبنان.
